In [1]:
import numpy as np
from torchvision import transforms
import torch
import torch.nn as nn
from torch import optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Dataset
import time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [2]:
class CustomDataset(Dataset): 
    def __init__(self):
        self.x_data = np.random.randint(1,100,size=(10000,2))
        #self.x_data = np.concatenate([self.x_data, self.x_data**2], axis = -1)
        self.y_data = np.sqrt(self.x_data[:,-1]**2 + self.x_data[:,-2]**2)
        self.y_data = self.y_data.reshape(-1,1)

  # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y
    
train_dataset = CustomDataset()
print(train_dataset.__getitem__(0))
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size = batch_size)
total_len = len(train_loader)

(tensor([34., 89.]), tensor([95.2733]))


In [3]:
class MNISTNet(nn.Module):
    
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.linear1 = nn.Linear(2,64)
        self.linear2 = nn.Linear(64,1)
        self.linear3 = nn.Linear(4,1)
        
    def forward(self, X):
        '''
        func = [torch.sqrt, torch.sigmoid,torch.tanh]
        out = F.relu(self.linear1(X))
        
        new_out = torch.zeros(out.shape)
        for i in range(3):
            new_out[i] = func[i](out[i])
            '''
        out = torch.relu(self.linear1(X))
        return self.linear2(out)

def train(model, train_set, optimizer1, criterion, total_len):
    total_cost = 0
    l = np.linspace(0,total_len,10)
    l_idx = 0
    for batch_idx, (data, target) in enumerate(train_set):
        if batch_idx >= l[l_idx]:
            print('-', end='')
            l_idx+=1
        optimizer1.zero_grad()
        output = model(data.to(device))
        loss = criterion(output, target.to(device))
        total_cost += loss
        loss.backward()
        optimizer1.step()
    print('>', end='')
    print('    loss :',total_cost.item(), end='\n')

model = MNISTNet()

In [6]:

optimizer1 = optim.Adam(model.parameters(), lr = 0.0001)
#optimizer2 = optim.Adam(model.linear2(), lr = 0.005)
criterion = nn.MSELoss()
for epoch in range(20):
    train(model, train_loader, optimizer1, criterion, total_len)

--------->    loss : 0.6725348234176636
--------->    loss : 0.5960959196090698
--------->    loss : 0.5627841353416443
--------->    loss : 0.5264521241188049
--------->    loss : 0.49640953540802
--------->    loss : 0.47023630142211914
--------->    loss : 0.4465523064136505
--------->    loss : 0.425555020570755
--------->    loss : 0.4074619710445404
--------->    loss : 0.3918375074863434
--------->    loss : 0.3777654767036438
--------->    loss : 0.36477962136268616
--------->    loss : 0.3527860641479492
--------->    loss : 0.3414049446582794
--------->    loss : 0.3308000862598419
--------->    loss : 0.3207862973213196
--------->    loss : 0.3117416501045227
--------->    loss : 0.30344873666763306
--------->    loss : 0.2958031892776489
--------->    loss : 0.2888300120830536


In [8]:
model.linear1.weight

Parameter containing:
tensor([[-0.5227, -0.2993],
        [-0.5438, -0.5445],
        [-0.2920, -0.1460],
        [-0.3656,  0.4551],
        [-0.3003,  0.1572],
        [-0.1821,  0.3038],
        [ 0.4412, -0.0764],
        [ 0.0038,  0.4060],
        [ 0.3617,  0.2714],
        [ 0.5324,  0.3585],
        [-0.7865,  0.1911],
        [ 0.6071,  0.2643],
        [ 0.4126, -0.7363],
        [-0.5669, -0.0430],
        [-0.3795, -0.6892],
        [-0.5446,  0.2007],
        [-0.4093,  0.0067],
        [-0.1745,  0.5521],
        [ 0.7463,  0.3766],
        [ 0.2530, -0.5248],
        [-0.2588, -0.0946],
        [ 0.2255, -0.2929],
        [ 0.5103, -0.5507],
        [-0.4249, -0.6645],
        [ 0.6538, -0.6098],
        [-0.5030,  0.3887],
        [ 0.2756,  0.6308],
        [ 0.6383,  0.7118],
        [ 0.5586, -0.4196],
        [ 0.3965,  0.0634],
        [-0.1060, -0.1940],
        [-0.2356, -0.0215],
        [-0.6681,  0.3624],
        [ 0.3963,  0.7138],
        [ 0.3690, -0.0027]

In [9]:
import torch.nn.functional as F

model.eval()
x = 1000000
y = 20
output = model(torch.FloatTensor([x,y]))
print(output, np.sqrt(x**2+y**2))

tensor([997345.5625], grad_fn=<AddBackward0>) 1000000.0002
